In [1]:
import numpy as np
import pandas as pd

In [3]:
train_df = pd.read_csv("E:\VIL Confidential Information Dataset\Train_imputed.csv")

In [4]:
test_df = pd.read_csv("E:\VIL Confidential Information Dataset\Test_imputed.csv")

In [62]:
cleaned_test_df = pd.read_csv("E:\VIL Codefest\secret\VIL Confidential Information Dataset\Test.csv")

In [5]:
train_df.head()

,Unnamed: 0,Age,ARPU,Age on Network,Brand Identifier,Circle Name,Connection Type,Data Usage,Gender,Genre,International Usage,Recharge,SMS Usage,VAS Subscription,Voice Usage,Web/App,DND
0,0,1,1,2,1,2,1,2,1,2,0,6,6,43,5,1,0
1,1,4,1,6,1,2,1,6,1,3,0,3,1,87,7,1,0
2,2,2,2,7,0,0,1,2,1,3,0,2,1,87,8,1,1
3,3,9,7,1,1,3,1,7,0,2,1,4,1,87,5,1,1
4,4,9,4,3,1,2,0,2,3,3,0,3,3,43,5,1,0


In [6]:
internet_new = {
    0: 0,
    6: 1,
    1: 2,
    3: 3, 
    4: 4,
    5: 5,
    2: 6,
    7: 7    
}

sms_new = {
    0: 0,
    1: 1,
    3: 2,
    6: 3, 
    2: 4,
    4: 5,
    5: 6,
    7: 7    
}

voice_new = {
    0: 0,
    1: 1,
    6: 2,
    2: 3,
    3: 4,
    4: 5,
    5: 6,
    7: 7,
    8: 8
}

arpu_new = {
    0: 0,
    1: 1,
    6: 2,
    2: 3,
    3: 4,
    4: 5,
    5: 6,
    7: 7
}

recharge_new = {
    0: 0,
    1: 1,
    8: 2,
    2: 3,
    3: 4,
    4: 5,
    5: 6,
    6: 7,
    7: 8,
    9: 9,
    10: 10
}

new_age = {
    0: '18-23',
    1: '24-28',
    2: '29-33',
    3: '34-38',
    4: '39-43',
    5: '44-48',
    6: '49-53',
    7: '54-58',
    8: '<18',
    9: '>58'
}

# Frontend Data

In [7]:
internet_usage = test_df[['Data Usage']].replace(internet_new)
sms_usage = test_df[['SMS Usage']].replace(sms_new)
voice_usage = test_df[['Voice Usage']].replace(voice_new)
arpu = test_df[['ARPU']].replace(arpu_new)
recharge = test_df[['ARPU']].replace(recharge_new)

In [8]:
from sklearn.preprocessing import MinMaxScaler

In [9]:
def scaling(arr):
    scaler = MinMaxScaler(feature_range=(0, 1000))
    arr = scaler.fit_transform(arr)
    arr = pd.DataFrame(arr).apply(lambda x: round(x)).astype('int')
    arr = np.asanyarray(arr)
    return arr

In [10]:
internet_usage = np.ravel(scaling(internet_usage))
sms_usage = np.ravel(scaling(sms_usage))
voice_usage = np.ravel(scaling(voice_usage))
arpu = np.ravel(scaling(arpu))
recharge = np.ravel(scaling(recharge))
age = np.array(test_df['Age'].replace(new_age))
location = np.array(test_df['Circle Name'].replace({0: 'Delhi', 1: 'Kerala', 2: 'Kolkata', 3: 'Mumbai'}))
gender = np.array(test_df['Gender'].replace({0:'F', 1:'M', 2:'O'}))

In [11]:
age

array(['>58', '39-43', '39-43', ..., '29-33', '49-53', '24-28'],
      dtype=object)

In [12]:
frontend_dict = {
    'Internet Usage': internet_usage,
    'SMS Usage': sms_usage,
    'Voice Usage': voice_usage,
    'ARPU': arpu,
    'Recharge': recharge,
    'Age': age,
    'Location': location,
    'Gender': gender
}

In [13]:
for k, v in frontend_dict.items():
    print(len(v))

48242
48242
48242
48242
48242
48242
48242
48242


In [14]:
frontend_df = pd.DataFrame.from_dict(frontend_dict)
frontend_df['Churn'] = 0
frontend_df['Customer Segment'] = 0

# Churn Data

In [15]:
from random import *

In [16]:
churn = []
for i in range(frontend_df.shape[0]):
    val1 = (frontend_df.iloc[i]['Internet Usage'])
    val2 = (frontend_df.iloc[i]['SMS Usage'])
    val3 = (frontend_df.iloc[i]['Voice Usage'])
    val4 = (frontend_df.iloc[i]['ARPU'])
    values = [val1, val2, val3, val4]
    count = 0
    for val in values:
        if val<500:
            count += 1
    if count == 4:
        churn.append(randint(80, 100))
    elif count == 3:
        churn.append(randint(60, 80))
    elif count == 2:
        churn.append(randint(40, 60))
    elif count == 1:
        churn.append(randint(20, 40))
    else:
        churn.append(randint(0, 20))

In [17]:
frontend_df['Churn'] = churn

In [18]:
frontend_df['Churn'].value_counts()

40    1501
20    1045
60    1007
22     957
26     934
      ... 
5      122
7      120
10     120
3      115
17     113
Name: Churn, Length: 101, dtype: int64

In [19]:
frontend_df.head()

,Internet Usage,SMS Usage,Voice Usage,ARPU,Recharge,Age,Location,Gender,Churn,Customer Segment
0,143,143,125,143,125,>58,Kolkata,M,85,0
1,857,143,375,429,375,39-43,Kolkata,F,76,0
2,714,143,1000,857,750,39-43,Kerala,M,22,0
3,429,0,375,1000,1000,39-43,Kerala,M,71,0
4,857,143,1000,1000,1000,>58,Kerala,M,35,0


In [20]:
frontend_df.to_csv(r"E:\VIL Confidential Information Dataset\frontend.csv", index=False)